# US Hurricane Catastrophe Model

A very simple model of US hurricane exposure parameterized to observed event frequency 1851-2017 and a smattering of losses 1970-2017.

* Frequency: Poisson(1.74)
* Severity: lognormal($\mu=19.595, \sigma=2.581$)

## Model Algorithm I 

    for year = 1 to N
        simulate number of events E from Poisson(1.74)
        for event_num = 1 to E
            simulate loss L from Lognormal(mu, sigma)
            store year, event_num, L

## Model Algorithm II 

This method is more suited to a spreadsheet implementation. 

    time = 0
    event_id = 0
    last_year = 0
    for event = 1 to N
        simulate waiting time for event t from Exponential(1.74)
        time = time + t
        year = integer part of time
        if year > last_year then event_id = 0, last_year = year
        simulate loss L from Lognormal(mu, sigma)
        store year, event_id, L
        event_id = event_id + 1


In [ ]:
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline
sns.set(context='paper', style='darkgrid', font='sans')

## Claim Count Component

Based on an analysis of landfalling hurricanes since 1851 we selected a Poisson(1.74) to model the number of hurricanes making landfall in the Continental US each year. 

The next few commands create a variable to represent this random variable. 

In [ ]:
freq = 290 / 167
poi = ss.poisson(freq)

In [ ]:
# simulate 10 random draws
poi.rvs(10)

In [ ]:
# generate 100,000 draws and creates a histogram
N = 100000
temp = poi.rvs(N)
plt.hist(temp, bins=np.arange(12), density=True)
plt.title('Histogram of Model Frequency');

In [ ]:
# can compute frequencies by hand 
unq = np.unique(temp, return_counts=True)
unq

In [ ]:
# can compare with actual 
model = [np.exp(-freq)]
for i in range(1,len(unq[0])):
    model.append(model[i-1] * freq / i)
model = np.array(model)

In [ ]:
# compare with freqs
print('    N Simulation           Model         Error')
for i, s, m in zip(unq[0], unq[1], N*model):
    print(f'{i:5d}\t{s:8,d}\t{m:8,.1f}\t{s/m-1:6.3f}')


In [ ]:
# or the same thing using pandas
df = pd.DataFrame(dict(N=unq[0], Simulation=unq[1]/N, Model=model))
df['Error'] = df.Simulation / df.Model - 1
df = df.set_index("N")
df.index.name="N"
df

In [ ]:
df[['Model', 'Simulation']].plot(kind='bar')

In [ ]:
df.loc[:, ['Model', 'Simulation']].plot(kind='bar')

## Severity Component

Lognormal(mu, sigma)


In [ ]:
mu = 19.595
sigma = 2.581

In [ ]:
# variable to simulate lognormals
ln = ss.lognorm(sigma, scale=np.exp(mu))

In [ ]:
# mean and variance
ln.stats('mv')

In [ ]:
sample = ln.rvs(N)

In [ ]:
# compare sample mean with known mean of lognormal
np.array((sample.mean(), np.exp(mu + sigma**2 / 2), ln.stats('m'))) / 1e9

In [ ]:
# histogram of log losses 
plt.hist(np.log(sample), density=True, bins=30);

In [ ]:
# probability plot = perfect
ss.probplot(np.log(sample), plot=plt)

# Aggregate Losses

## Model Algorithm I 

    for year = 1 to N
        simulate number of events E from Poisson(1.74)
        for event_num = 1 to E
            simulate loss L from Lognormal(mu, sigma)
            store year, event_num, L

In [ ]:
# setup  variables 
# N = number of simulation years
N = 100000
freq = 290 / 167
freq_dist = ss.poisson(freq)
mu = 19.595
sigma = 2.581
sev_dist = ss.lognorm(sigma, scale=np.exp(mu))

In [ ]:
# simulate annual event founds 
event_counts = freq_dist.rvs(N)
num_events = event_counts.sum()
num_events, N * freq

## ELT and YLT

Event Loss Table, ELT, will have columns

* Event ID
* Simulated Year
* Event Loss

Yearly Loss Table, YLT, will have columns 

* Year
* Event count, the number of events in the year
* Sum = total loss in the year (for Agg PMLs)
* Max = largest loss in the year (for occ PMLs)

In [ ]:
# container for YLT
ylt = np.zeros((N, 4))

In [ ]:
# fill in year ID and event count, arrays are 0 based
ylt[:, 0] = np.arange(N)
ylt[:, 1] = event_counts

In [ ]:
# look at the top ten rows (:10) and all columns (:)
ylt[:10,:]

In [ ]:
# container for ELT, now we know number of rows - sum of num events from YLT
elt = np.zeros((num_events, 3))
# add event ID, counter 0, 1, ...
elt[:, 0] = np.arange(num_events)

In [ ]:
# how big is ylt: should be freq x N rows, can use shape or len:
elt.shape,  len(elt), freq * N, freq, num_events, N

In [ ]:
# simulate individual event losses
elt[:, 2] = sev_dist.rvs(num_events)

In [ ]:
# look at answer
elt[:10, :]

In [ ]:
# tricky part...need to fill in the event years in the YLT
# make an array showing starting and ending number of events for each year
event_boundaries = event_counts.cumsum()
# start at zero
event_boundaries = np.hstack((0, event_boundaries))

In [ ]:
ylt[:10, 1], event_boundaries[:10]

In [ ]:
# non pythonic 
%%timeit 
for i in range(N):
    if event_boundaries[i] < event_boundaries[i+1]:
        elt[event_boundaries[i]:event_boundaries[i+1], 1] = i

In [ ]:
# num events per year and the created year IDs
ylt[:10,1], elt[:10, :]

In [ ]:
# maybe more pythonic
# %%timeit
for i, (b, e) in enumerate(zip(event_boundaries[:-1], event_boundaries[1:])):
    if e > b:
        elt[b:e, 1] = i

In [ ]:
# top 10 rows of ylt and elt
ylt[:10, :], elt[:10, :]

In [ ]:
# combine: add losses to ELT and summarize back to the YLT, non pythonic
# %%timeit
for i in range(N-1):
    if event_boundaries[i] < event_boundaries[i+1]:
        elt[event_boundaries[i]:event_boundaries[i+1], 1] = i
        ylt[i, 2] = elt[event_boundaries[i]:event_boundaries[i+1], 2].sum()
        ylt[i, 3] = elt[event_boundaries[i]:event_boundaries[i+1], 2].max()

In [ ]:
# top 10 rows of ylt and elt
ylt[:10, :], elt[:10, :]

In [ ]:
# combine: add losses to ELT and summarize back to the YLT, pythonic
# %%timeit
for i, (b, e) in enumerate(zip(event_boundaries[:-1], event_boundaries[1:])):
    if e > b:
        elt[b:e, 1] = i
        ylt[i, 2] = elt[b:e, 2].sum()
        ylt[i, 3] = elt[b:e, 2].max()

In [ ]:
# top 10 rows of ylt and elt
ylt[:10, :], elt[:10, :]

### Use data types and name columns

In [ ]:
def simulate1(N, freq, mu, sigma):
    """
    Simulate ELT and YLT with N years, Poisson(freq) and lognormal(mu, sigma)
    severity
    
    """
    freq_dist = ss.poisson(freq)
    sev_dist = ss.lognorm(sigma, scale=np.exp(mu))

    # simulate events per year, figure number of events 
    event_counts = freq_dist.rvs(N)
    num_events = event_counts.sum()
    num_events, N * freq
    
    # make YLT
    ylt = np.zeros(N, dtype=[('year_id', 'int32'), ('num_events', 'int32'), 
                         ('sum_loss', 'float64'), 
                         ('max_loss', 'float64')])
    
    # fill in year ID and event count, arrays are 0 based
    ylt['year_id'] = np.arange(N)
    ylt['num_events'] = event_counts
    
    # container for ELT, now we know number of rows - sum of num events from YLT
    elt = np.zeros(num_events, dtype=[('event_id', 'int32'), ('year_id', 'int32'), 
                         ('loss', 'float64')])
    # add event ID, counter 0, 1, ...
    elt['event_id'] = np.arange(num_events)
       
    event_boundaries = event_counts.cumsum()
    event_boundaries = np.hstack((0, event_boundaries))
    elt['loss'] = sev_dist.rvs(num_events)
    
    # combine: add losses to ELT and summarize back to the YLT, non pythonic
    for i in range(N-1):
        if event_boundaries[i] < event_boundaries[i+1]:
            elt[event_boundaries[i]:event_boundaries[i+1]]['year_id'] = i
            ylt[i]['sum_loss'] = elt[event_boundaries[i]:event_boundaries[i+1]]['loss'].sum()
            ylt[i]['max_loss'] = elt[event_boundaries[i]:event_boundaries[i+1]]['loss'].max()
            
    # sort YLT by loss 
    ylt.sort(order='sum_loss')
    
    # return answer
    return elt, ylt

In [ ]:
%timeit elt, ylt = simulate1(100000, 290/167, mu, sigma)

In [ ]:
def simulate2(N, freq, mu, sigma):
    """
    Simulate ELT and YLT with N years, Poisson(freq) and lognormal(mu, sigma)
    severity. Alternative aggregation to YLT.

    """
    freq_dist = ss.poisson(freq)
    sev_dist = ss.lognorm(sigma, scale=np.exp(mu))

    # simulate events per year, figure number of events 
    event_counts = freq_dist.rvs(N)
    num_events = event_counts.sum()
    num_events, N * freq
    
    # make YLT
    ylt = np.zeros(N, dtype=[('year_id', 'int32'), ('num_events', 'int32'), 
                         ('sum_loss', 'float64'), 
                         ('max_loss', 'float64')])
   
    # fill in year ID and event count, arrays are 0 based
    ylt['year_id'] = np.arange(N)
    ylt['num_events'] = event_counts
    
    # container for ELT, now we know number of rows - sum of num events from YLT
    elt = np.zeros(num_events, dtype=[('event_id', 'int32'), ('year_id', 'int32'), 
                         ('loss', 'float64')])
    # add event ID, counter 0, 1, ...
    elt['event_id'] = np.arange(num_events)
    
    
    event_boundaries = event_counts.cumsum()
    event_boundaries = np.hstack((0, event_boundaries))
    elt['loss'] = sev_dist.rvs(num_events)
    
    # combine: add losses to ELT and summarize back to the YLT, non pythonic
    for i in range(N-1):
        if event_boundaries[i] < event_boundaries[i+1]:
            temp = elt[event_boundaries[i]:event_boundaries[i+1]]
            temp['year_id'] = i
            ylt[i]['sum_loss'] = temp['loss'].sum()
            ylt[i]['max_loss'] = temp['loss'].max()
            
    # sort YLT by loss 
    ylt.sort(order='sum_loss')
    
    # return answer
    return elt, ylt

In [ ]:
%timeit elt, ylt = simulate2(100000, 290/167, mu, sigma)

In [ ]:
def simulate3(num_events, freq, mu, sigma):
    """
    Simulate ELT and YLT with num_events events, Poisson(freq) and lognormal(mu, sigma)
    severity. Method II exponential waiting time approach. Use Pandas.

    """
    
    freq_dist = ss.poisson(freq)
    sev_dist = ss.lognorm(sigma, scale=np.exp(mu))

    # simulate event years
    e = ss.expon(scale=1/freq)
    event_times = np.array(e.rvs(num_events).cumsum(), dtype=int)
    N = event_times[-1]

    # container for ELT, given number of rows 
    elt = np.zeros(num_events, dtype=[('event_id', 'int32'), ('year_id', 'int32'), 
                         ('loss', 'float64')])
    elt = pd.DataFrame(elt)
    # add event ID, counter 0, 1, ...
    elt['event_id'] = np.arange(num_events)
    elt['year_id'] = event_times
    elt['loss'] = sev_dist.rvs(num_events)
    elt = elt.set_index('event_id')
    
    # make YLT
    ylt = np.zeros(N, dtype=[('year_id', 'int32'), ('num_events', 'int32'), 
                         ('sum_loss', 'float64'), 
                         ('max_loss', 'float64')])
    ylt = pd.DataFrame(ylt)
    # fill in year ID and event count, arrays are 0 based
    ylt['year_id'] = np.arange(N)
    ylt = ylt.set_index('year_id')
    
    g = elt.groupby('year_id')['loss'].agg([np.sum, np.max, np.size])   
    ylt['sum_loss'] = g['sum']
    ylt['max_loss'] = g['amax']
    ylt['num_events'] = g['size']
    ylt = ylt.fillna(0)
    
    # sort YLT by loss 
    ylt = ylt.sort_values('sum_loss', ascending=False)
    
    # return answer
    return elt, ylt

In [ ]:
%timeit elt, ylt = simulate3(100000, 290/167, mu, sigma)

In [ ]:
elt, ylt = simulate3(100000, 290/167, mu, sigma)

In [ ]:
ylt.head()

In [ ]:
elt.head()

In [ ]:
ylt[['sum_loss', 'max_loss']].reset_index(drop=True).plot(logy=True, subplots=True, figsize=(6,10));

In [ ]:
plt.hist( np.log(ylt.max_loss / ylt.sum_loss), bins=100 );